# Fault Injection on Quantized Neural Networks

This notebook tests fault injection on quantized neural networks (QNNs). It is recommended to first read through and understand some of the example notebooks that demonstrate image classification with BNN-PYNQ.

## 1. Import the packages

In [1]:
import bnn
import os
import csv
from copy import deepcopy
from json import dumps
from PIL import Image
from yapf.yapflib.yapf_api import FormatCode
import scipy.io as sio
import matplotlib.pyplot as plt
from pynq import Xlnk

## 2. Load the required datasets

This notebook utilizes the CIFAR-10, GTSRB, SVHN, and MNIST datasets. You can download them from each given url via wget and unzip it to a folder on Pynq as shown below.

#### CIFAR-10

In [2]:
def load_cifar10_testset(folder, num_images=10000):
    num_images = min(num_images, 10000)

    input_file = ''
    if num_images == 10000:
        input_file = folder + "/test_batch.bin"
    else:
        input_file = folder + "/test_batch_{}".format(num_images) + ".bin"

    if not os.path.exists(input_file):
        with open(folder + "/test_batch.bin", "rb") as infile:
            with open(input_file, "wb+") as outfile:
                for i in range(num_images):
                    outfile.write(infile.read(3073))

    labels = []
    with open(input_file, "rb") as file:
        for i in range(num_images):
            #read first byte -> label
            labels.append(int.from_bytes(file.read(1), byteorder="big"))
            #read image (3072 bytes) and do nothing with it
            file.read(3072)
        file.close()

    return (input_file, labels)


if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/cifar-10-binary.tar.gz"):
    !wget https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz

if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/cifar-10-batches-bin/"):
    !tar -xf cifar-10-binary.tar.gz

cifar_files, cifar_labels = load_cifar10_testset("/home/xilinx/jupyter_notebooks/bnn/cifar-10-batches-bin/", 1000)

#### GTSRB

In [3]:
def load_gtsrb_testset(folder, num_images=12630):
    num_images = min(num_images, 12630)

    image_files = []
    images = []
    labels = []

    with open(folder + "/GT-final_test.csv") as gtfile:
        gtreader = csv.reader(gtfile, delimiter=';')
        next(gtreader)
        for row in gtreader:
            image_files.append(folder + '/' + row[0])
            labels.append(int(row[7]))

    labels = labels[0:num_images]
    for i in range(num_images):
        images.append(Image.open(image_files[i]))
        images[i].load()

    return (images, labels)


if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/GTSRB_Final_Test_Images.zip"):
    !wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_Images.zip

if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/GTSRB_Final_Test_GT.zip"):
    !wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_GT.zip

if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/GTSRB"):
    !unzip -q -o GTSRB_Final_Test_Images.zip
    !unzip -q -o GTSRB_Final_Test_GT.zip
    !mv GT-final_test.csv GTSRB/Final_Test/Images

gtsrb_files, gtsrb_labels = load_gtsrb_testset("/home/xilinx/jupyter_notebooks/bnn/GTSRB/Final_Test/Images", 1000)

#### SVHN

In [4]:
def load_svhn_testset(file, num_images=26032):
    num_images = min(num_images, 26032)

    images = []
    labels = []

    data = sio.loadmat(file)

    # Subtact 1 to match classifier output. E.g. The classifier will return class 1 for an image of the number 2.
    label_mat = data['y'] - 1
    image_mat = data['X']

    labels = label_mat.transpose().tolist()[0]
    labels = labels[0:num_images]

    #for i in range(image_mat.shape[3]):
    for i in range(num_images):
        images.append(Image.fromarray(image_mat[:,:,:,i]))
        images[i].load()

    return (images, labels)


if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/test_32x32.mat"):
    !wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat

svhn_files, svhn_labels = load_svhn_testset("/home/xilinx/jupyter_notebooks/bnn/test_32x32.mat", 1000)

#### MNIST

In [5]:
def load_mnist_testset(folder, num_images=10000):
    num_images = min(num_images, 10000)

    idx3_path = folder + "/t10k-images-idx3-ubyte"
    idx1_path = folder + "/t10k-labels-idx1-ubyte"
    labels = []
    
    with open(idx1_path, "rb") as lbl_file:
        #read magic number and number of labels (MSB first) -> MNIST header
        magicNum = int.from_bytes(lbl_file.read(4), byteorder="big")
        countLbl = int.from_bytes(lbl_file.read(4), byteorder="big")
        #now the labels are following byte-wise
        for idx in range(num_images):
            labels.append(int.from_bytes(lbl_file.read(1), byteorder="big"))
        lbl_file.close()

    return (idx3_path, labels)
    
if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/t10k-images-idx3-ubyte.gz"):
    !wget http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz 
    !wget http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz 
        
if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/t10k-images-idx3-ubyte"):
    !gzip -d t10k-images-idx3-ubyte.gz
    !gzip -d t10k-labels-idx1-ubyte.gz

mnist_files, mnist_labels = load_mnist_testset("/home/xilinx/jupyter_notebooks/bnn/")

--2020-02-25 11:45:11--  http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘yann.lecun.com’
--2020-02-25 11:45:12--  http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘yann.lecun.com’


## 3. Start Xlnk Interface

In [6]:
xlnk = Xlnk()

## 4. Define Utility Functions

Various functions for handling dictionaries and calculating statistics

In [7]:
def calculate_accuracy(results, labels):
    countRight = 0
    for idx in range(len(labels)):
        if labels[idx] == results[idx]:
            countRight += 1
    return countRight*100/len(labels)


# Merge dictionaries that contain sub-dictionaries
def dict_of_dicts_merge(*dicts):
    out = {}
    for item in dicts:
        overlapping_keys = out.keys() & item.keys()
        for key in overlapping_keys:
            if (isinstance(out[key], dict) and isinstance(item[key], dict)):
                out[key] = dict_of_dicts_merge(out[key], item[key])
        for key in item.keys() - overlapping_keys:
            out[key] = deepcopy(item[key])
    return out


def dict_to_str(dict):
    dict_string = dumps(dict)
    formatted_code, _ = FormatCode(dict_string)
    return formatted_code


def write_dict_to_file(file_name, output_dict):
    os.makedirs(os.path.dirname(file_name), exist_ok=True)
    f = open(file_name, "w+")
    f.write(dict_to_str(output_dict))
    f.close()

&nbsp;
## 5. A Simple Fault Test Example
This example demonstrates the most simple method of running a fault test, and demonstrates the use of all of the classification function arguments. The classes beyond this section expand on this concept and allow the user to more easily set up customizeable and comprehensive tests.

The test is performed in a way very simlar to how a normal image classification is performed. The main difference is that `classifier.classify_cifars_with_faults()` is called instead of `classifier.classify_cifars()`. This function takes additional arguments that specify the type, location, and number of faults.

For the CnvClassifier, non-CIFAR10 datasets (e.g. GTSRB and SVHN) will need to be converted to CIFAR-10 format before being classified. This will be automatically handled by using the `classifier.classify_images()` function instead of `classifier.classify_cifars()`. This function takes a list of Pillow Images and converts them to the required format before classifying. For fault injection with these datasets, use `classifier.classify_images_with_faults()`.

The inference can be performed with different precision for weights and activation. Creating a specific Classifier will automatically download the correct bitstream onto PL and load the weights and thresholds trained on the specific dataset.

In [8]:
print("Availabe params:", bnn.available_params(bnn.NETWORK_CNVW1A1))

# Instantiate the cnvW1A1 hardware classifier with the CIFAR10 dataset
classifier = bnn.CnvClassifier(bnn.NETWORK_CNVW1A1, 'cifar10', bnn.RUNTIME_HW)

# Classify the dataset, and inject 10000 MBUs, targeting both weights and thresholds in the first 3 layers.
#
# The default value of target_layers is an empty list, which will target all layers.
#
# target_type determines if weights or thresholds will be targeted. A value of -1 causes both to be targeted, a value
# of 0 causes weights to be targeted, and a value of 1 will target thresholds.
print("Classifying", len(cifar_labels), "images with", classifier.net + "-" + classifier.params)
results = classifier.classify_cifars_with_faults(cifar_files, num_faults=10000, flip_word=True, target_type=-1, target_layers=[0, 1, 2])

# Reset the device
xlnk.xlnk_reset()

# Calculate the accuracy of the network
accuracy = calculate_accuracy(results, cifar_labels)

print("Accuracy:", accuracy)

Availabe params: ['cifar10', 'road-signs', 'streetview']


/usr/local/lib/python3.6/dist-packages/pynq/overlay.py:299: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


Classifying 1000 images with cnvW1A1-cifar10


/usr/local/lib/python3.6/dist-packages/bnn/bnn.py:179: UserWarning: implicit cast from 'char *' to a different pointer type: will be forbidden in the future (check that the types are as you expect; use an explicit ffi.cast() if they are correct)
  path.encode(), len(self.classes), size_ptr, usecperimage, num_faults, 1 if flip_word else 0, target_type, targets, len(target_layers)


Inference took 1582000.00 microseconds, 1582.00 usec per image
Classification rate: 632.11 images per second
Accuracy: 61.8


&nbsp;

## 6. Define Fault Testing Classes

### Fault Test Class
Uses the given network type and image set to run a series of fault tests on the network. The network is reset between runs.
CNVFaultTest and LFCFaultTest provide class methods for easier instantiation of test with a specific classifier and dataset.

**Constructor Arguments:**
- **classifier_cls:** The classifier class type to use (e.g. bnn.CnvClassifier, bnn.LfcClassifier)
- **network:** The network to test. e.g. bnn.NETWORK_CNVW1A1
- **dataset:** The dataset that's being tested. (`'cifar10'`, `'streetview'`, or `'road-signs'`)
- **input_file:** The file, or list of Images for non-cifar tests, containing the images to be tested.
- **labels:** The list of ground truth labels for calculating accuracy.
- **control_accuracy:** Optional parameter specifying the control accuracy of the network. Will be calculated if not provided.

**FaultTest.run_test() Arguments:**
- **num_runs:** The number of tests to perform
- **num_flips:** The number of faults to inject per run
- **flip_word:** A boolean indicating if a bit or word should be flipped
- **weight_or_threshold:** An integer specifying if weights, thresholds, or both should be targeted. Obtain these values using the static methods in the TargetType class for clarity.

        -1 = target weights and thresholds
        0  = target weights only
        1  = target thresholds only

- **target_layers:** An array of integers specifying which layers to target. Leave empty to target all layers.

In [9]:
class TargetType:
    @staticmethod
    def any():
        return -1

    @staticmethod
    def weights():
        return 0

    @staticmethod
    def thresholds():
        return 1


class FaultTest:
    def __init__(self, classifier_cls, network, dataset, input_file, labels):
        self.classifier_cls = classifier_cls
        self.network = network
        self.dataset = dataset
        self.input_file = input_file
        self.labels = labels
    
    def run_test(self, num_runs, num_flips, flip_word=False, target_type=TargetType.any(), target_layers=[]):
        results    = [ None for i in range(num_runs) ]
        times      = [ None for i in range(num_runs) ]
        accuracies = [ None for i in range(num_runs) ]

        for i in range(num_runs):
            classifier = self.classifier_cls(self.network, self.dataset, bnn.RUNTIME_HW)

            message = "{}-{} run {} of {} (flipping {}{} {}(s) in {})".format(
                self.network,
                self.dataset,
                i+1,
                num_runs,
                num_flips,
                ' weight' if target_type == 0 else ' threshold' if target_type == 1 else '',
                'word' if flip_word else 'bit',
                'any layer' if len(target_layers) == 0 else 'layer(s) {}'.format(target_layers)
            )
            print(message)

            if self.dataset == 'cifar10':
                results[i] = classifier.classify_cifars_with_faults(self.input_file, num_flips, flip_word, target_type, target_layers).tolist()
            elif self.dataset == 'mnist':
                results[i] = classifier.classify_mnists_with_faults(self.input_file, num_flips, flip_word, target_type, target_layers).tolist()
            else:
                results[i] = classifier.classify_images_with_faults(self.input_file, num_flips, flip_word, target_type, target_layers).tolist()

            times[i]      = classifier.usecPerImage
            accuracies[i] = calculate_accuracy(results[i], self.labels)

            print("Accuracy:", accuracies[i])

            xlnk.xlnk_reset()
            print()

        return (results, times, accuracies)


class CNVFaultTest(FaultTest):
    def __init__(self, network, dataset, input_file, labels):
        super().__init__(bnn.CnvClassifier, network, dataset, input_file, labels)

    @classmethod
    def CIFARTest(cls, network, input_file, labels):
        return cls(network, 'cifar10', input_file, labels)

    @classmethod
    def SVHNTest(cls, network, input_file, labels):
        return cls(network, 'streetview', input_file, labels)

    @classmethod
    def GTSRBTest(cls, network, input_file, labels):
        return cls(network, 'road-signs', input_file, labels)


class LFCFaultTest(FaultTest):
    def __init__(self, network, dataset, input_file, labels):
        super().__init__(bnn.LfcClassifier, network, dataset, input_file, labels)

    @classmethod
    def MNISTTest(cls, network, input_file, labels):
        return cls(network, 'mnist', input_file, labels)

&nbsp;

### Network Test Class
Takes a FaultTest object and use it to run a comprehensive series of tests on different combinations of targets and SEU/MBU

**Constructor Arguments:**
- **fault_test:** A FaultTest (or derived class) object
- **output_folder:** The folder to store the test results in. Results will be stored in subfolders organized by network and dataset.

**NetworkTest.test_network() Arguments:**
- **num_runs:** The number of runs in each test
- **flip_counts:** A list of fault counts to inject each run. One test will be run for each.
- **test_types:** A list of TestType objects which specify the location and type of fault for each test.
- **target_layers:** An array of integers specifying which layers to target. Leave empty to target all layers.

In [ ]:
class TestType:
    def __init__(self, target_type, flip_word):
        self.target_type = target_type
        self.flip_word = flip_word
        self.__build_name()

    def __build_name(self):
        self.name = ''

        if self.target_type == TargetType.any():
            self.name += 'any'
        elif self.target_type == TargetType.weights():
            self.name += 'weight'
        else:
            self.name += 'threshold'

        self.name += ' '

        if self.flip_word:
            self.name += 'word'
        else:
            self.name += 'bit'

    @classmethod
    def any_bit(cls):
        return cls(TargetType.any(), False)

    @classmethod
    def any_word(cls):
        return cls(TargetType.any(), True)

    @classmethod
    def weight_bit(cls):
        return cls(TargetType.weights(), False)

    @classmethod
    def weight_word(cls):
        return cls(TargetType.weights(), True)

    @classmethod
    def threshold_bit(cls):
        return cls(TargetType.thresholds(), False)

    @classmethod
    def threshold_word(cls):
        return cls(TargetType.thresholds(), True)



class NetworkTest:
    def __init__(self, fault_test, output_folder):
        self.fault_test = fault_test
        self.output_folder = output_folder + '/' + self.fault_test.network + '/' + self.fault_test.dataset + '/'
        self.control = None

    def __run_control(self):
        print("Running", self.fault_test.network + "-" + self.fault_test.dataset, "control test")
        _, _, accuracy = self.fault_test.run_test(num_runs=1, num_flips=0);
        self.control = accuracy[0]

    # Output dict contains the combined raw results from a test. Used as input to calculate_stats
    def __build_output_dict(self, name, num_runs, num_flips, layers, accuracies):
        out = {}
        out["network"] = self.fault_test.network
        out["dataset"] = self.fault_test.dataset
        out["run count"] = num_runs
        out["flips"] = num_flips
        out["control"] = self.control
        out["layers"] = layers
        out["results"] = {}
        out["results"][name] = accuracies
        return out

    # Calculates various stats given the results of a test (formatted with __build_output_dict)
    def __calculate_stats(self, output_dict):
        out = output_dict.copy()
        for key, value in output_dict["results"].items():
            out["results"][key] = {}
            out["results"][key]["runs"] = {}
            out["results"][key]["runs"]["all"] = value
            out["results"][key]["runs"]["effective"] = list(filter(lambda x: x != out["control"], value))
            out["results"][key]["effective count"] = len(out["results"][key]["runs"]["effective"])
            out["results"][key]["min accuracy"] = min(value)
            out["results"][key]["max accuracy"] = max(value)
            if out["results"][key]["effective count"] != 0:
                out["results"][key]["avg accuracy"] = sum(value) / len(value)
                out["results"][key]["avg effective accuracy"] = sum(out["results"][key]["runs"]["effective"]) / out["results"][key]["effective count"]
                out["results"][key]["accuracy delta"] = out["control"] - out["results"][key]["avg accuracy"]
                out["results"][key]["effective accuracy delta"] = out["control"] - out["results"][key]["avg effective accuracy"]
            else:
                out["results"][key]["avg accuracy"] = out["control"]
        return out


    def __run_tests(self, folder, num_runs, num_flips, test_types, target_layers):
        output_dicts = []

        for test in test_types:
            _, _, accuracies = self.fault_test.run_test(num_runs, num_flips, test.flip_word, test.target_type, target_layers)
            output = self.__build_output_dict(test.name, num_runs, num_flips, target_layers, accuracies)
            output_dicts.append(output)
            write_dict_to_file(folder + '/temp/' + self.fault_test.network + '_results' + test.name + '.json', output)

        # Calculate stats
        stats = dict_of_dicts_merge(*output_dicts)
        stats = self.__calculate_stats(stats)
        return stats

        
    def test_network(self, num_runs, flip_counts, test_types, target_layers=[]):
        # Calculate the control accuracy
        if self.control is None:
            self.__run_control()

        for num_flips in flip_counts:
            folder = self.output_folder + '/' + str(num_flips) + 'flips/'

            # Run all tests for this flip count
            stats = self.__run_tests(folder, num_runs, num_flips, test_types, target_layers)

            # Build file name
            filename = folder + '/' + self.fault_test.network + '_' + self.fault_test.dataset
            if len(target_layers) > 0:
                filename = filename + "_stats_layer{}.json".format(target_layers)
            else:
                filename = filename + "_stats.json"

            # Write results to the output file
            write_dict_to_file(filename, stats)


    def comprehensive_test(self, num_runs, flip_counts, target_layers=[]):
        test_types = [
            TestType.any_bit(), TestType.any_word(),
            TestType.weight_bit(), TestType.weight_word(),
            TestType.threshold_bit(), TestType.threshold_word()
        ]

        self.test_network(num_runs, flip_counts, test_types, target_layers)

&nbsp;

## 7. Run the tests
The dictionary below instantiates tests for all of the available CNV networks. These tests are then conducted for each individual layer multiple times, with an increasing number of faults each time.

In [ ]:
output_folder = "/home/xilinx/jupyter_notebooks/bnn/faults/"

num_runs = 50
flip_counts = [5, 10, 50, 100]

cnv_tests = {
    # cnvW1A1 Networks
    'cnvW1A1': [
#        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A1, cifar_files, cifar_labels), output_folder),
#        NetworkTest(CNVFaultTest.SVHNTest(bnn.NETWORK_CNVW1A1, svhn_files, svhn_labels), output_folder),
#        NetworkTest(CNVFaultTest.GTSRBTest(bnn.NETWORK_CNVW1A1, gtsrb_files, gtsrb_labels), output_folder),
    ],
    'cnvW1A1-TMR': [
#        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A1_TMR, cifar_files, cifar_labels), output_folder),
#        NetworkTest(CNVFaultTest.SVHNTest(bnn.NETWORK_CNVW1A1_TMR, svhn_files, svhn_labels), output_folder),
#        NetworkTest(CNVFaultTest.GTSRBTest(bnn.NETWORK_CNVW1A1_TMR, gtsrb_files, gtsrb_labels), output_folder),
    ],
    'cnvW1A1-interleaved': [
#        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A1_INTERLEAVED, cifar_files, cifar_labels), output_folder),
#        NetworkTest(CNVFaultTest.SVHNTest(bnn.NETWORK_CNVW1A1_INTERLEAVED, svhn_files, svhn_labels), output_folder),
#        NetworkTest(CNVFaultTest.GTSRBTest(bnn.NETWORK_CNVW1A1_INTERLEAVED, gtsrb_files, gtsrb_labels), output_folder),
    ],

    # cnvW1A2 Networks
    'cnvW1A2': [
#        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A2, cifar_files, cifar_labels), output_folder)
    ],
    'cnvW1A2-TMR': [
#        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A2_TMR, cifar_files, cifar_labels), output_folder)
    ],
    'cnvW1A2-interleaved': [
        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A2_INTERLEAVED, cifar_files, cifar_labels), output_folder)
    ],

    # cnvW2A2 Networks
    'cnvW2A2': [
#        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW2A2, cifar_files, cifar_labels), output_folder)
    ],
    'cnvW2A2-TMR': [
#        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW2A2_TMR, cifar_files, cifar_labels), output_folder)
    ],
    'cnvW2A2-interleaved': [
        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW2A2_INTERLEAVED, cifar_files, cifar_labels), output_folder)
    ],
}

In [ ]:
all_types = [TestType.weight_bit(), TestType.weight_word(), TestType.threshold_bit(), TestType.threshold_word()]
only_weights = [TestType.weight_bit(), TestType.weight_word()]

all_tests_flat = [test for lst in cnv_tests.values() for test in lst]

for test in all_tests_flat:
    test.test_network(num_runs, flip_counts, test_types=all_types, target_layers=[])